Notebook qui définit les tables dans databricks.

databases_to_set: liste des bases de données à définir.

In [0]:
%run ./Utils/COMMON

In [0]:
dbutils.widgets.multiselect(
    name='databases_to_set',
    defaultValue='all',
    choices=[
        'all',
        'GROUP_REFERENTIAL',
        'QAD', 
        "DELTA_MASTER", 
        "PACKAGER",
        "WEB",
        "ORDER_MANAGEMENT"
    ],
    label='Databases To Set'
)

if any(',' in c for c in dbutils.widgets.get('databases_to_set')):
    databases_to_set = dbutils.widgets.get('databases_to_set').split(",")
else:
    databases_to_set = [dbutils.widgets.get('databases_to_set')]

In [0]:
import csv
import io

databases = ['GROUP_REFERENTIAL', 'QAD', "DELTA_MASTER", "PACKAGER", "WEB", "ORDER_MANAGEMENT"]
errors = []

for database in databases:
    if (any(database.lower() == d.lower() for d in databases_to_set)) or (any('all' == d.lower() for d in databases_to_set)):
        if database == "WEB":
            tables = [{'input': 'WEB', 'output': 'eurofxref'}]
        else:
            # Lecture du fichier de configuration
            tables = csv.DictReader(io.StringIO(dbutils.fs.head(f"dbfs:/mnt/dlk/{CONFIG_CONTAINER}/{database}/params.csv")), delimiter=';')
        # Création de la base de données
        spark.sql(f"CREATE DATABASE IF NOT EXISTS {database}")
        # Création des tables
        for table in tables:
            table_name = f"{database}.{table.get('output')}"
            table_name = table_name.replace('-', '_')
            try:
                spark.sql(f"DROP TABLE IF EXISTS {table_name}")
                spark.sql(f"CREATE TABLE {table_name} USING DELTA LOCATION '/mnt/dlk/silver/{database}/{table.get('output')}'")
            except:
                errors.append(table_name)

if len(errors) == 0:
    dbutils.notebook.exit('All tables are created.')             
else:
    dbutils.notebook.exit('Unable to create: ' + ','.join(errors) + '.')             

All tables are created.